# Araç Tipi Sınıflandırma

### Hazırlayanlar;

Aysu Çetin, 
Ali Haydar Kaplan, 
Alper Tiryaki

Projemiz 2020 Ocak ayını kapsayıp 31 veri setinden üzerinden oluşturulmuştur. Ancak temsili olması dolayısıyla bu sayfada sadece ilk gün uygulamasını yapacağız.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
import pygeohash as pgh

In [ ]:
df = pd.read_csv("OneDrive/Masaüstü/ats/ats_01_01.csv", sep = ';' ,header = 'infer')

In [ ]:
df[["XCOORD","YCOORD","RAWSPEED","RAWDATE","SEGID","VEHICLEID"]] = df['XCOORD,"YCOORD","RAWSPEED","RAWDATE","SEGID","VEHICLEID"'].str.split(",", expand = True)

df.drop(columns ='XCOORD,"YCOORD","RAWSPEED","RAWDATE","SEGID","VEHICLEID"', inplace = True )

In [ ]:
df[['YCOORD', 'XCOORD']] = df[['XCOORD', 'YCOORD']]

In [ ]:
df.columns = ('YCOORD', 'XCOORD','RAWSPEED', 'RAWDATE', 'SEGID', 'VEHICLEID')

## Eksik Veri Analizi

X ve Y koordinatları kullanılarak Coğrafi Bilgi Sistemlerinden geohash değerlerini oluşturulacağız ve köprülerden geçiş bilgilerine erişeceğiz. Bu yüzden eksik veri analizi için öncelikli olarak XCOORD ve YCOORD değişkenlerini inceliyoruz.

In [ ]:
df["XCOORD"].isna().value_counts()

In [ ]:
df["YCOORD"].isna().value_counts()

In [ ]:
df['geohash']=df.apply(lambda x: pgh.encode(float(x.YCOORD), float(x.XCOORD), precision=6), axis=1)

* 15 Temmuz Şehitler Köprüsü

Başlangıç:

In [ ]:
pgh.encode(41.043356, 29.036328, precision=6)

Bitiş:

In [ ]:
pgh.encode(41.048013, 29.031479, precision=6)

* Fatih Sultan Mehmet Köprüsü

Başlangıç:

In [ ]:
pgh.encode(41.091459, 29.065769, precision=6)

In [ ]:
pgh.encode(41.091572, 29.065426, precision=6)

Bitiş:

In [ ]:
pgh.encode(41.091038, 29.057261, precision=6)

In [ ]:
pgh.encode(41.091224, 29.057433, precision=6)

* Yavuz Sultan Selim Köprüsü

Başlangıç:

In [ ]:
pgh.encode(41.200106, 29.118037, precision=6)

In [ ]:
pgh.encode(41.200605, 29.117578, precision=6)

Bitiş:

In [ ]:
pgh.encode(41.205194, 29.106605, precision=6)

In [ ]:
pgh.encode(41.205309, 29.107039, precision=6)

In [ ]:
filt15gs = ((df["geohash"] == "sxk9kx") | (df["geohash"] == "sxk9s8"))
df_15 = df[filt15gs]

df_15["konum"] = "15temmuz"

In [ ]:
filtfsm = ((df["geohash"] == "sxk9v2") | (df["geohash"] == "sxk9v0"))
df_fsm = df[filtfsm]

df_fsm["konum"] = "fsm"

In [ ]:
filtyvz = ((df["geohash"] == "sxkdqs") | (df["geohash"] == "sxkdqm"))
df_yvz = df[filtyvz]

df_yvz["konum"] = "yavuzsultanselim"

In [ ]:
df_k = pd.DataFrame(np.concatenate((df_15, df_fsm, df_yvz) , 
                                   axis = 0), 
                                columns = ('YCOORD', 'XCOORD','RAWSPEED', 'RAWDATE', 'SEGID', 'VEHICLEID', 'geohash', 'konum'))

In [ ]:
df_k

In [ ]:
df_k["VEHICLEID"] = df_k["VEHICLEID"].str.strip('"')

In [ ]:
%%time
dfs = {}

for e in df_k["VEHICLEID"].unique():
    filt = (df_k["VEHICLEID"] == e)
    fknm = df_k[filt]["konum"]
    
    if np.any(fknm == "15temmuz"):
        dfs[e] = {"VEHICLEID": e, "type": "hafif araç"}
    
    elif np.any(fknm == "fsm"):
        dfs[e] = {"VEHICLEID": e, "type": "hafif araç"}
    
    elif np.all(fknm == "yavuzsultanselim"):
        dfs[e] = {"VEHICLEID": e,"type": "ağır araç"}

In [ ]:
dfs = pd.DataFrame(dfs)
dfs = dfs.T

In [ ]:
dfs["type"].value_counts()

In [ ]:
dfs